In [1]:
import pandas as pd

df=pd.read_csv('/Users/putuu/Documents/Github_Repo/MMDT_2025_MLAI101/Khaing-Hsu-Wai/data/fraud.csv', index_col = 0)
y = df['Class'].values
df = df.iloc[:,1:]
X = df.drop(columns = 'Class').values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    test_size = 0.40, 
                                    random_state=1)

In [3]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

,priors,None
,var_smoothing,1e-09


In [12]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train, y_train)


ValueError: Negative values in data passed to MultinomialNB (input X).

In [15]:
import numpy as np

# Checking negative value for MultinomialNB
print("neg in X train-", np.sum(X_train < 0))
print("neg in y train-", np.sum(y_train < 0))

neg in X train- 183310
neg in y train- 0


In [5]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB(alpha=1.0)
bnb.fit(X_train, y_train)

/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


,alpha,1.0
,force_alpha,True
,binarize,0.0
,fit_prior,True
,class_prior,None


In [11]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

results = []

# --- 1) GaussianNB ---
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_g = gnb.predict(X_test)

print("=== GaussianNB classification report ===")
print(classification_report(y_test, y_pred_g, digits=4))
results.append({
    "Model": "GaussianNB",
    "Precision (1)": precision_score(y_test, y_pred_g, pos_label=1, zero_division=0),
    "Recall (1)":    recall_score(y_test, y_pred_g,    pos_label=1, zero_division=0),
    "F1 (1)":        f1_score(y_test, y_pred_g,        pos_label=1, zero_division=0)
})

# --- 2) BernoulliNB ---
bnb = BernoulliNB()  # default alpha=1.0, binarize=0.0
bnb.fit(X_train, y_train)
y_pred_b = bnb.predict(X_test)

print("\n=== BernoulliNB classification report ===")
print(classification_report(y_test, y_pred_b, digits=4))
results.append({
    "Model": "BernoulliNB",
    "Precision (1)": precision_score(y_test, y_pred_b, pos_label=1, zero_division=0),
    "Recall (1)":    recall_score(y_test, y_pred_b,    pos_label=1, zero_division=0),
    "F1 (1)":        f1_score(y_test, y_pred_b,        pos_label=1, zero_division=0)
})

# --- 3) MultinomialNB (make non-negative "counts") ---
mms = MinMaxScaler()
X_train_m = mms.fit_transform(X_train)
X_test_m  = mms.transform(X_test)
# Convert to small integer "counts" to stabilize MNB
X_train_m = np.rint(np.clip(X_train_m, 0, None) * 100).astype(int)
X_test_m  = np.rint(np.clip(X_test_m, 0, None) * 100).astype(int)

mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train_m, y_train)
y_pred_m = mnb.predict(X_test_m)

print("\n=== MultinomialNB classification report ===")
print(classification_report(y_test, y_pred_m, digits=4))
results.append({
    "Model": "MultinomialNB",
    "Precision (1)": precision_score(y_test, y_pred_m, pos_label=1, zero_division=0),
    "Recall (1)":    recall_score(y_test, y_pred_m,    pos_label=1, zero_division=0),
    "F1 (1)":        f1_score(y_test, y_pred_m,        pos_label=1, zero_division=0)
})

# --- Plain-text summary table (no .style / jinja2 needed) ---
df_metrics = pd.DataFrame(results).sort_values(
    by=["Recall (1)", "Precision (1)"], ascending=False
).reset_index(drop=True)

print("\n=== Precision/Recall/F1 for Class 1 (fraud) ===")
print(df_metrics.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


=== GaussianNB classification report ===
              precision    recall  f1-score   support

           0     0.9977    0.9775    0.9875      8541
           1     0.3806    0.8613    0.5280       137

    accuracy                         0.9757      8678
   macro avg     0.6892    0.9194    0.7577      8678
weighted avg     0.9880    0.9757    0.9803      8678


=== BernoulliNB classification report ===
              precision    recall  f1-score   support

           0     0.9961    0.9991    0.9976      8541
           1     0.9286    0.7591    0.8353       137

    accuracy                         0.9953      8678
   macro avg     0.9624    0.8791    0.9165      8678
weighted avg     0.9951    0.9953    0.9950      8678


=== MultinomialNB classification report ===
              precision    recall  f1-score   support

           0     0.9958    0.9995    0.9977      8541
           1     0.9619    0.7372    0.8347       137

    accuracy                         0.9954      8678

/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/putuu

In [13]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Utility to evaluate a model
def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision (1)": precision_score(y_test, y_pred, pos_label=1, zero_division=0),
        "Recall (1)": recall_score(y_test, y_pred, pos_label=1, zero_division=0),
        "F1 (1)": f1_score(y_test, y_pred, pos_label=1, zero_division=0)
    }

results = []

# --- GaussianNB (continuous features) ---
gnb = GaussianNB()
results.append(evaluate_model("GaussianNB", gnb, X_train, y_train, X_test, y_test))

# --- BernoulliNB (binary features) ---
bnb = BernoulliNB()
results.append(evaluate_model("BernoulliNB", bnb, X_train, y_train, X_test, y_test))

# --- MultinomialNB (requires non-negative counts) ---
mms = MinMaxScaler()
X_train_m = mms.fit_transform(X_train)
X_test_m  = mms.transform(X_test)

# Convert scaled values to integer "counts"
X_train_counts = np.rint(X_train_m * 100).astype(int)
X_test_counts  = np.rint(X_test_m * 100).astype(int)

mnb = MultinomialNB(alpha=1.0)
results.append(evaluate_model("MultinomialNB", mnb, X_train_counts, y_train, X_test_counts, y_test))

# --- Summary table ---
df_results = pd.DataFrame(results)
print("\n=== Naive Bayes Models: Precision/Recall for Fraud (Class 1) ===")
print(df_results.to_string(index=False, float_format=lambda x: f"{x:.3f}"))




=== Naive Bayes Models: Precision/Recall for Fraud (Class 1) ===
        Model  Accuracy  Precision (1)  Recall (1)  F1 (1)
   GaussianNB     0.976          0.381       0.861   0.528
  BernoulliNB     0.995          0.929       0.759   0.835
MultinomialNB     0.995          0.962       0.737   0.835


/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/putuu/Documents/Github_Repo/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/putuu